In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('weather_data.csv')

In [19]:
df.head()

,time,temperature_2m,precipitation,relativehumidity_2m,windspeed_10m,winddirection_10m,surface_pressure,cloudcover,city
0,2026-01-17 00:00:00,7.4,0.1,91,9.7,129,1008.9,100,Edinburgh
1,2026-01-17 01:00:00,7.7,0.1,91,8.9,133,1009.0,100,Edinburgh
2,2026-01-17 02:00:00,7.9,0.1,91,7.9,133,1009.3,100,Edinburgh
3,2026-01-17 03:00:00,7.8,0.0,91,8.4,133,1008.7,100,Edinburgh
4,2026-01-17 04:00:00,7.8,0.0,91,7.4,133,1009.6,100,Edinburgh


In [20]:
df = df.rename(columns={'temperature_2m':'temperature','relativehumidity_2m': 'relative_humidity','windspeed_10m':'wind_speed','winddirection_10m':'wind_direction'})

In [21]:
df.shape

(174720, 9)

In [9]:
df.isna().sum()

time                 0
temperature          0
precipitation        0
relative_humidity    0
wind_speed           0
wind_direction       0
surface_pressure     0
cloudcover           0
city                 0
dtype: int64

In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.describe()

,temperature,precipitation,relative_humidity,wind_speed,wind_direction,surface_pressure,cloudcover
count,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000
mean,8.273214,0.100595,87.142857,10.002976,130.404762,1003.048214,93.714286
std,1.188363,0.247968,5.791313,4.015094,31.769206,11.431886,10.231367
min,4.600000,0.000000,70.000000,2.700000,59.000000,986.500000,51.000000
25%,7.500000,0.000000,84.000000,6.800000,113.750000,989.675000,92.750000
50%,8.300000,0.000000,88.000000,9.050000,133.500000,1008.800000,98.000000
75%,8.900000,0.100000,91.000000,13.250000,151.250000,1012.825000,100.000000
max,10.900000,1.200000,96.000000,19.400000,193.000000,1017.800000,100.000000


In [12]:
df['temp_1h'] = df['temperature'].shift(1)  
df['temp_2h'] = df['temperature'].shift(2)  

In [14]:
df.head()

,time,temperature,precipitation,relative_humidity,wind_speed,wind_direction,surface_pressure,cloudcover,city,temp_1h,temp_2h
0,2026-01-17 00:00:00,7.4,0.1,91,9.7,129,1008.9,100,Edinburgh,NaN,NaN
1,2026-01-17 01:00:00,7.7,0.1,91,8.9,133,1009.0,100,Edinburgh,7.4,NaN
2,2026-01-17 02:00:00,7.9,0.1,91,7.9,133,1009.3,100,Edinburgh,7.7,7.4
3,2026-01-17 03:00:00,7.8,0.0,91,8.4,133,1008.7,100,Edinburgh,7.9,7.7
4,2026-01-17 04:00:00,7.8,0.0,91,7.4,133,1009.6,100,Edinburgh,7.8,7.9


In [16]:
df.isna().sum()
df.dropna()

,time,temperature,precipitation,relative_humidity,wind_speed,wind_direction,surface_pressure,cloudcover,city,temp_1h,temp_2h
2,2026-01-17 02:00:00,7.9,0.1,91,7.9,133,1009.3,100,Edinburgh,7.7,7.4
3,2026-01-17 03:00:00,7.8,0.0,91,8.4,133,1008.7,100,Edinburgh,7.9,7.7
4,2026-01-17 04:00:00,7.8,0.0,91,7.4,133,1009.6,100,Edinburgh,7.8,7.9
5,2026-01-17 05:00:00,7.7,0.0,91,7.9,133,1009.7,96,Edinburgh,7.8,7.8
6,2026-01-17 06:00:00,7.8,0.0,89,7.4,151,1010.2,99,Edinburgh,7.7,7.8
...,...,...,...,...,...,...,...,...,...,...,...
174715,2006-02-24 19:00:00,6.1,0.0,89,6.5,161,995.5,97,Edinburgh,6.6,7.3
174716,2006-02-24 20:00:00,5.7,0.0,91,6.0,163,996.4,95,Edinburgh,6.1,6.6
174717,2006-02-24 21:00:00,5.3,0.0,93,5.6,165,997.1,92,Edinburgh,5.7,6.1
174718,2006-02-24 22:00:00,4.9,0.0,94,4.7,157,997.7,76,Edinburgh,5.3,5.7


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174720 entries, 0 to 174719
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   time               174720 non-null  object 
 1   temperature        174720 non-null  float64
 2   precipitation      174720 non-null  float64
 3   relative_humidity  174720 non-null  int64  
 4   wind_speed         174720 non-null  float64
 5   wind_direction     174720 non-null  int64  
 6   surface_pressure   174720 non-null  float64
 7   cloudcover         174720 non-null  int64  
 8   city               174720 non-null  object 
dtypes: float64(4), int64(3), object(2)
memory usage: 12.0+ MB


In [ ]:
df['time']=pd.to_datetime(df['time'])

In [32]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer


In [30]:
x = df.drop('precipitation', axis=1)
y = df['precipitation']

In [33]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,)

In [35]:
num_cols = x.select_dtypes(include=['float64','int64']).columns.tolist()
cat_cols = x.select_dtypes(include=['object']).columns.tolist()

In [34]:
preprocessor = ColumnTransformer([('num',StandardScaler(),num_cols),('cat',OneHotEncoder(),cat_cols)])
x_train_transformed = preprocessor.fit_transform(x_train)
x_test_transformed = preprocessor.fit(x_test)




ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [29]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso